In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
import category_encoders as ce
import xgboost as xgb
from datetime import datetime

# TRAIN

In [2]:
train = pd.read_csv(r'C:\Users\rvuon\OneDrive\Documents\GitHub\githubtest\Data\train.csv')

In [3]:
# train = pd.read_csv(r'/Users/kimberly/Documents/GitHub/githubtest/Data/train.csv')

## FillNA Functions

In [4]:
def fillna_groupby_train(df, feature, group, agg_funct):
    df[feature] = df.groupby(group)[feature].transform(lambda x: x.fillna(x.agg(agg_funct).iloc[0]))
    return df

def fillna_mode_groupby_test(test, feature, group):
    mode_dict = train.groupby(group)[feature].apply(lambda x: x.mode().iloc[0]).to_dict()
    test.loc[test[feature].isnull(), feature] = test[group].map(mode_dict)
    return test

# def fill_with_mode_or_default(test, feature, group):
#     mode_dict = train.groupby(group)[feature].apply(lambda x: x.mode().iloc[0]).to_dict()
#     test.loc[test[feature].isnull(), feature] = test[group].map(mode_dict)
#     return test


def fillna_mean_groupby_test(test, train, feature, group):
    test[feature] = train.groupby(group)[feature].transform(lambda x: x.fillna(x.mean()))
    return test

# def fillna_mode_groupby_test(test, train, feature, group):
#     def mode_func(x):
#         return x.mode().iloc[0]  # Define a custom function to calculate the mode for each group

#     fill_value = train.groupby(group)[feature].transform(mode_func)  # Calculate the mode for each group
#     test[feature] = test[feature].fillna(fill_value)  # Fill missing values in the test DataFrame
#     return test

# def fillna_groupby_test(test, train, feature, group, agg_funct):
#     return test.assign(**{feature: test[feature].fillna(train.groupby(group)[feature].transform(agg_funct))})

def fillna_NA(df, feature):
    df[feature] = np.where(df[feature].isnull(), 'NA', df[feature])
    return df

def fillna_ZERO(df, feature):
    df[feature] = np.where(df[feature].isnull(), 0, df[feature])
    return df

def fillna__NA_basedon_feat_equalzero(df, depend_feat, feature):
    df[feature] = np.where((df[feature].isnull()) & (df[depend_feat]==0), 'NA', df[feature])
    return df

def fillna__NA_basedon_feat_greaterzero(df, depend_feat, feature):
    df[feature] = np.where((df[feature].isnull()) & (df[depend_feat]> 0), 'NA', df[feature])
    return df

def fillna__value_basedon_feat_greaterzero(df, depend_feat, feature, value):
    df[feature] = np.where((df[feature].isnull()) & (df[depend_feat]> 0), value, df[feature])
    return df

def fillna__mode(df, feature):
    df[feature] = np.where(df[feature].isnull(), df[feature].mode(), df[feature])
    return df

def fillna_mean(df, feature):
    df[feature] = np.where(df[feature].isnull(), df[feature].mean(), df[feature])
    return df

def fillna_mode_test(test, train, feature):
    test[feature] = np.where(test[feature].isnull(), train[feature].mode(), test[feature])
    return test

def fillna_ZERO_basedon_feat_equalzero(df, depend_feat, feature):
    df[feature] = np.where((df[feature].isnull()) & (df[depend_feat]==0), 0, df[feature])
    return df

def fillna_otherfeat(df, depend_feat, feature):
    df[feature] = np.where(df[feature].isnull(), df[depend_feat], df[feature])
    return df
 
# FillNA Specific Features
def fillna_MiscFeature(df):
    df['MiscFeature'] = np.where((df['MiscFeature'].isnull()) & (df['GarageType']=='2Types'), 'Gar2', df['MiscFeature'])
    df['MiscFeature'] = np.where(df['MiscFeature'].isnull(), 'NA', df['MiscFeature'])
    return df

def fillna_BsmtFinType2(df):
    df['BsmtFinType2'] = np.where((df['BsmtFinType2'].isnull() & df['BsmtUnfSF'] > 0), 'Unf', df['BsmtFinType2'])
    df['BsmtFinType2'] = np.where(df['BsmtFinType2'].isnull(), 'NA', df['BsmtFinType2'])
    return df


## Feature Engineering Functions

In [5]:
def feateng_age(df, feat, newfeat):
    df[newfeat] = datetime.now().year - df[feat]
    return df

def feateng_ratio(df, feat1, feat2, newfeat):
    df[newfeat] = (df[feat1]/df[feat2]).round(2)
    return df

def feateng_has_YN(df, feat, newfeat):
    df[newfeat]= np.where(df[feat]=='Y', 1, 0)
    return df

def feateng_has_greaterzero(df, feat, newfeat):
    df[newfeat] = df[feat].apply(lambda x: 1 if x > 0 else 0)
    return df

## Transformation Functions

In [6]:
def log_feat(df):
    skewed_feats = df.skew(numeric_only=True)[df.skew(numeric_only=True) > 0.75]
    for feat in skewed_feats.index:
        df[feat] = np.log1p(df[feat])

## Datatype Functions

In [7]:
def convert_dtype(df, feat, dtype):
    df[feat] = df[feat].astype(dtype)
    return 

def convert_datetime_year(df, feat):
    df[feat] = pd.to_datetime(df[feat], format='%Y').dt.year
    return df

def convert_datetime_month(df, feat):
    df[feat] = pd.to_datetime(df[feat], format='%m').dt.month
    return df

def convert_datetime_number_year(df, feat):
    df[feat] = pd.to_datetime(df[feat], format='%Y').dt.strftime('%Y')
    return df

def convert_datetime_number_month(df, feat):
    df[feat] = pd.to_datetime(df[feat], format='%m').dt.strftime('%m')
    return df

def convert_float_int(df, feat):
    df[feat] = df[feat].astype(int)

## Encoding/Scaling Functions

In [8]:
def ordinal_encoding(df, ord_feats):
    ord = OrdinalEncoder()
    ord.fit(ord_feats)
    ord_feats_encoded = ord.transform(ord_feats)
    new_ord_feats = pd.DataFrame(ord_feats_encoded, columns=ord_feats.columns)
    df.drop(columns=ord_feats, inplace=True)
    df = df.merge(new_ord_feats, left_index=True, right_index=True)
    return df

def nom_encoding(df, nom_feats):
    one = ce.OneHotEncoder(use_cat_names=True)
    one.fit(nom_feats)
    nom_feats_encoded = one.transform(nom_feats)
    df.drop(columns=nom_feats, inplace=True)
    df = df.merge(nom_feats_encoded, left_index=True, right_index=True)
    return df

def cont_scaling(df, cont_feats):
    scaler_minmax = MinMaxScaler()
    scaler_minmax.fit(cont_feats)
    cont_feats_scaled = scaler_minmax.transform(cont_feats)
    new_cont_feats = pd.DataFrame(cont_feats_scaled, columns=cont_feats.columns)
    df.drop(columns=cont_feats, inplace=True)
    df = df.merge(new_cont_feats, left_index=True, right_index=True)
    return df

## Outliers

In [9]:
train = train.drop(train[(train['GrLivArea'] > 4000) & (train['SalePrice'] < 200000)].index).reset_index(drop=True)

## Fill Null Features

In [10]:

fillna_groupby_train(train, 'LotFrontage', 'Neighborhood', 'mode')
fillna_NA(train, ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'Fence'])
fillna_ZERO(train, 'MasVnrArea')
fillna__NA_basedon_feat_equalzero(train, 'TotalBsmtSF', 'BsmtExposure')
fillna__NA_basedon_feat_equalzero(train, 'Fireplaces', 'FireplaceQu')
fillna__NA_basedon_feat_equalzero(train, 'GarageCars', 'GarageType')
fillna_ZERO_basedon_feat_equalzero(train, 'GarageCars', 'GarageYrBlt')
fillna__NA_basedon_feat_equalzero(train, 'GarageCars', 'GarageFinish')
fillna__NA_basedon_feat_equalzero(train, 'GarageCars', 'GarageQual')
fillna__NA_basedon_feat_equalzero(train, 'GarageCars', 'GarageCond')
fillna__NA_basedon_feat_equalzero(train, 'PoolArea', 'PoolQC')
fillna__mode(train, ['Electrical', 'BsmtExposure'])
fillna_BsmtFinType2(train)
fillna_MiscFeature(train)
train.isnull().sum().sort_values(ascending=False)

Id             0
CentralAir     0
GarageYrBlt    0
GarageType     0
FireplaceQu    0
              ..
MasVnrArea     0
MasVnrType     0
Exterior2nd    0
Exterior1st    0
SalePrice      0
Length: 81, dtype: int64

## Feature Engineering 

In [11]:
feateng_age(train, 'YearBuilt', 'age')
feateng_age(train, 'YearRemodAdd', 'remodel_age')
feateng_ratio(train, 'GrLivArea', 'LotArea', 'livingtolot')
feateng_has_YN(train, 'PavedDrive', 'has_paveddrive')
feateng_has_greaterzero(train, 'Fireplaces', 'has_fireplace')
feateng_has_greaterzero(train, 'PoolArea', 'has_pool')


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,YrSold,SaleType,SaleCondition,SalePrice,age,remodel_age,livingtolot,has_paveddrive,has_fireplace,has_pool
0,1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,...,2008,WD,Normal,208500,21,21,0.20,1,0,0
1,2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,...,2007,WD,Normal,181500,48,48,0.13,1,1,0
2,3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,...,2008,WD,Normal,223500,23,22,0.16,1,1,0
3,4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,...,2006,WD,Abnorml,140000,109,54,0.18,1,1,0
4,5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,...,2008,WD,Normal,250000,24,24,0.15,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1456,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,...,2007,WD,Normal,175000,25,24,0.21,1,1,0
1454,1457,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,...,2010,WD,Normal,210000,46,36,0.16,1,1,0
1455,1458,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,...,2010,WD,Normal,266500,83,18,0.26,1,1,0
1456,1459,20,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,...,2010,WD,Normal,142125,74,28,0.11,1,0,0


## Transformation

In [12]:
log_feat(train)

## Datatype conversions

In [13]:
convert_dtype(train, ['MSSubClass', 'OverallCond', 'GarageFinish'], 'str')
convert_dtype(train, ['LotFrontage', 'MasVnrArea'], 'int')
convert_datetime_year(train, 'YearBuilt')
convert_datetime_year(train, 'YearRemodAdd')
convert_datetime_month(train, 'MoSold')
convert_datetime_number_year(train, 'YrSold')
convert_float_int(train, 'GarageYrBlt')

## Encoding/Scaling

In [14]:
df = train.copy()

In [15]:
ord_feats = df[[
'LotShape',
'LandContour',
'Utilities',
'LandSlope',
'OverallQual',
'OverallCond',
'YearBuilt',
'YearRemodAdd',
'ExterQual',
'ExterCond',
'BsmtQual',
'BsmtCond',
'BsmtExposure',
'BsmtFinType1',
'BsmtFinType2',
'HeatingQC',
'CentralAir',
'KitchenQual',
'Functional',
'FireplaceQu',
'GarageYrBlt',
'GarageFinish',
'GarageQual',
'GarageCond',
'PoolQC',
'Fence',
'MiscVal'
]]

nom_feats = df[[
'MSSubClass',
'MSZoning',
'Street',
'Alley',
'LotConfig',
'Neighborhood',
'Condition1',
'Condition2',
'BldgType',
'HouseStyle',
'RoofStyle',
'RoofMatl',
'Exterior1st',
'Exterior2nd',
'MasVnrType',
'Foundation',
'Heating',
'Electrical',
'GarageType',
'PavedDrive',
'MiscFeature',
'SaleType',
'SaleCondition',

'has_paveddrive'
]]


cont_feats = df[[
'LotFrontage',
'LotArea',
'MasVnrArea',
'BsmtFinSF1',
'BsmtFinSF2',
'BsmtUnfSF',
'TotalBsmtSF',
'1stFlrSF',
'2ndFlrSF',
'LowQualFinSF',
'GrLivArea',
'BsmtFullBath',
'BsmtHalfBath',
'FullBath',
'HalfBath',
'BedroomAbvGr',
'KitchenAbvGr',
'TotRmsAbvGrd',
'Fireplaces',
'GarageCars',
'GarageArea',
'WoodDeckSF',
'OpenPorchSF',
'EnclosedPorch',
'3SsnPorch',
'ScreenPorch',
'PoolArea',
'MoSold',
'YrSold'


]]


In [16]:
ord = OrdinalEncoder()
ord.fit(ord_feats)
ord_feats_encoded = ord.transform(ord_feats)
new_ord_feats = pd.DataFrame(ord_feats_encoded, columns=ord_feats.columns)
df.drop(columns=ord_feats, inplace=True)
df = df.merge(new_ord_feats, left_index=True, right_index=True)

one = ce.OneHotEncoder(use_cat_names=True)
one.fit(nom_feats)
nom_feats_encoded = one.transform(nom_feats)
df.drop(columns=nom_feats, inplace=True)
df = df.merge(nom_feats_encoded, left_index=True, right_index=True)

scaler_minmax = MinMaxScaler()
scaler_minmax.fit(cont_feats)
cont_feats_scaled = scaler_minmax.transform(cont_feats)
new_cont_feats = pd.DataFrame(cont_feats_scaled, columns=cont_feats.columns)
df.drop(columns=cont_feats, inplace=True)
df = df.merge(new_cont_feats, left_index=True, right_index=True)

In [17]:
# ordinal_encoding(df, ord_feats)
# nom_encoding(df, nom_feats)
# cont_scaling(df, cont_feats)

## Train Test Split

In [18]:
y = df['SalePrice']
x = df.drop(columns=['Id', 'SalePrice'])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((1166, 242), (1166,), (292, 242), (292,))

## XGB

In [19]:
xgbr = xgb.XGBRegressor(n_estimators=1000, max_depth=2) #regularization and increase or reduce learning rate
xgbr.fit(xtrain, ytrain)

test_pred = xgbr.predict(xtest)
train_pred = xgbr.predict(xtrain)

ytrain = np.expm1(ytrain)
ytest = np.expm1(ytest)
test_pred = np.expm1(test_pred)
train_pred = np.expm1(train_pred)

print('RMSE Test:', mean_squared_error(ytest, test_pred)**0.5)
print('RMSE Train:', mean_squared_error(ytrain, train_pred)**0.5)

RMSE Test: 22360.360090242102
RMSE Train: 3437.7195765263814


# TEST

In [20]:
test = pd.read_csv(r'C:\Users\rvuon\OneDrive\Documents\GitHub\githubtest\Data\test.csv')

In [21]:
# test = pd.read_csv(r'/Users/kimberly/Documents/GitHub/githubtest/Data/test.csv')

In [22]:
test.isnull().sum().sort_values(ascending=False)

PoolQC           1456
MiscFeature      1408
Alley            1352
Fence            1169
MasVnrType        894
                 ... 
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
SaleCondition       0
Length: 80, dtype: int64

In [23]:
test['MSZoning'].isnull().sum()

4

In [24]:
test[test['MSZoning'].isnull()][['Neighborhood', 'MSZoning']]

,Neighborhood,MSZoning
455,IDOTRR,NaN
756,IDOTRR,NaN
790,IDOTRR,NaN
1444,Mitchel,NaN


In [25]:
train.groupby('Neighborhood')['MSZoning'].apply(lambda x: x.mode().iloc[0])

Neighborhood
Blmngtn    RL
Blueste    RM
BrDale     RM
BrkSide    RM
ClearCr    RL
CollgCr    RL
Crawfor    RL
Edwards    RL
Gilbert    RL
IDOTRR     RM
MeadowV    RM
Mitchel    RL
NAmes      RL
NPkVill    RL
NWAmes     RL
NoRidge    RL
NridgHt    RL
OldTown    RM
SWISU      RL
Sawyer     RL
SawyerW    RL
Somerst    FV
StoneBr    RL
Timber     RL
Veenker    RL
Name: MSZoning, dtype: object

In [26]:
test['MSZoning'].value_counts()

MSZoning
RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: count, dtype: int64

MSZoning
RL         1115
RM          245
FV           74
C (all)      15
RH           10

## Fill Null Features

In [27]:
fillna_mode_test(test, train, 'Neighborhood')
fillna_mode_groupby_test(test, 'MSZoning', 'Neighborhood')
fillna_mean_groupby_test(test, train, 'LotFrontage', 'Neighborhood')
fillna_NA(test, 'Alley')
fillna_mode_test(test, train, 'Utilities')
fillna_mode_test(test, train, 'Exterior1st')
fillna_mode_test(test, train, 'Exterior2nd')
fillna_NA(test, 'MasVnrType')
fillna_ZERO(test, 'MasVnrArea')
fillna_NA(test, 'BsmtQual')
fillna_NA(test, 'BsmtCond')
fillna__NA_basedon_feat_equalzero(test, 'TotalBsmtSF', 'BsmtExposure')
fillna_mode_test(test, train, 'BsmtExposure')
fillna_NA(test, 'BsmtFinType1')
fillna_ZERO(test, 'BsmtFinSF1')
fillna_BsmtFinType2(test)
fillna_NA(test, 'BsmtFinType2')
fillna_ZERO(test, 'BsmtFinSF2')
fillna_ZERO(test, 'BsmtUnfSF')
fillna_mode_groupby_test(test, 'Electrical', 'MSZoning')
fillna_ZERO(test, 'TotalBsmtSF')
fillna_ZERO(test, 'BsmtFullBath')
fillna_ZERO(test, 'BsmtHalfBath')
fillna_mode_groupby_test(test, 'KitchenAbvGr', 'KitchenQual')
fillna_mode_test(test, train, 'Functional')
fillna__NA_basedon_feat_equalzero(test, 'Fireplaces', 'FireplaceQu')
fillna__NA_basedon_feat_equalzero(test, 'GarageArea', 'GarageType')
fillna_ZERO_basedon_feat_equalzero(test, 'GarageArea', 'GarageYrBlt')
fillna_otherfeat(test, 'YearRemodAdd', 'GarageYrBlt')
fillna__NA_basedon_feat_equalzero(test, 'GarageArea', 'GarageFinish')
fillna_mode_test(test, train, 'GarageFinish')
fillna_mode_test(test, train, 'GarageCars')
fillna_mean(test, 'GarageArea')
fillna__NA_basedon_feat_equalzero(test, 'GarageArea', 'GarageQual')
fillna_mode_groupby_test(test, 'GarageFinish', 'GarageQual')
fillna__NA_basedon_feat_equalzero(test, 'GarageArea', 'GarageCond')
fillna_mode_groupby_test(test, 'GarageFinish', 'GarageCond')
fillna__NA_basedon_feat_equalzero(test, 'PoolArea', 'PoolQC')
fillna_mode_test(test, train, 'PoolQC')
fillna_NA(test, 'Fence')
fillna_MiscFeature(test)
fillna_NA(test, 'MiscFeature')
fillna_mode_test(test, train, 'SaleType')
test.isnull().sum().sort_values(ascending=False)





GarageQual       2
GarageCond       2
LotFrontage      1
KitchenQual      1
Id               0
                ..
MasVnrArea       0
MasVnrType       0
Exterior2nd      0
Exterior1st      0
SaleCondition    0
Length: 80, dtype: int64

In [30]:
test['MSZoning'].isnull().sum()

0

In [ ]:
test[test['Electrical']]

## Feature Engineering

In [38]:
feateng_age(test, 'YearBuilt', 'age')
feateng_age(test, 'YearRemodAdd', 'remodel_age')
feateng_ratio(test, 'GrLivArea', 'LotArea', 'livingtolot')
feateng_has_YN(test, 'PavedDrive', 'has_paveddrive')
feateng_has_greaterzero(test, 'Fireplaces', 'has_fireplace')
feateng_has_greaterzero(test, 'PoolArea', 'has_pool')


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MoSold,YrSold,SaleType,SaleCondition,age,remodel_age,livingtolot,has_paveddrive,has_fireplace,has_pool
0,1461,20,RH,4.0,11622,Pave,NA,Reg,Lvl,AllPub,...,6,2010,WD,Normal,63,63,0.08,1,0,0
1,1462,20,RL,4.0,14267,Pave,NA,IR1,Lvl,AllPub,...,6,2010,WD,Normal,66,66,0.09,1,0,0
2,1463,60,RL,4.0,13830,Pave,NA,IR1,Lvl,AllPub,...,3,2010,WD,Normal,27,26,0.12,1,1,0
3,1464,60,RL,4.0,9978,Pave,NA,IR1,Lvl,AllPub,...,6,2010,WD,Normal,26,26,0.16,1,1,0
4,1465,120,RL,4.0,5005,Pave,NA,IR1,HLS,AllPub,...,1,2010,WD,Normal,32,32,0.26,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,4.0,1936,Pave,NA,Reg,Lvl,AllPub,...,6,2006,WD,Normal,54,54,0.56,1,0,0
1455,2916,160,RM,4.0,1894,Pave,NA,Reg,Lvl,AllPub,...,4,2006,WD,Abnorml,54,54,0.58,1,0,0
1456,2917,20,RL,4.0,20000,Pave,NA,Reg,Lvl,AllPub,...,9,2006,WD,Abnorml,64,28,0.06,1,1,0
1457,2918,85,RL,4.0,10441,Pave,NA,Reg,Lvl,AllPub,...,7,2006,WD,Normal,32,32,0.09,1,0,0


## Datatype conversions

In [39]:
test['LotFrontage'].isnull().sum()

1

In [40]:
test['LotFrontage'].value_counts()

LotFrontage
4.0    1172
3.0     276
5.0      10
Name: count, dtype: int64

In [41]:
convert_dtype(test, ['MSSubClass', 'OverallCond'], 'str')
convert_dtype(test, ['LotFrontage', 'MasVnrArea'], 'int')
convert_datetime_year(test, 'YearBuilt')
convert_datetime_year(test, 'YearRemodAdd')
convert_datetime_month(test, 'MoSold')
convert_datetime_number_year(test, 'YrSold')
convert_float_int(test, 'GarageYrBlt')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer